# 06_cpu_svm_word_char.ipynb
## Modelo robusto: TF‑IDF (word + char) + Linear SVM

#Objetivo:

Construir um modelo robusto de classificação de fake news utilizando representações textuais híbridas (palavra + caractere) combinadas com um classificador linear, buscando melhorar o desempenho em relação ao baseline mantendo eficiência computacional e interpretabilidade.

#O que ele faz:

> 1. Pré-processamento dos dados:
- Construção do campo content = title + text
- Remoção de duplicatas completas
- Divisão estratificada em treino e validação

> 2. Extração de features:
- TF-IDF de palavras (conteúdo)
- TF-IDF de caracteres (estilo textual)

> 3. Treinamento do modelo Linear SVM:

- Ajuste de threshold com base no decision_function para otimização do F1-score

> 4. Avaliação do desempenho:

- Classification report
- F1-score no conjunto de validação

# Observação:

Mantém-se a decisão adotada no baseline de não utilizar as variáveis **subject** e **date**, a fim de evitar possível viés de fonte e sinais temporais artificiais no processo de aprendizagem. A abordagem baseada em n-grams de palavras e caracteres permite capturar simultaneamente padrões semânticos e estilísticos do texto, contribuindo para maior robustez do modelo sem aumento significativo do custo computacional.

In [1]:
# 0) Importação das bibliotecas e carregamento dos dados

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.pipeline import FeatureUnion

import joblib
import os

train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

In [2]:
## 1) Pré-processamento dos dados:

# content = title + text (sem subject)
train["content"] = train["title"].fillna("").astype(str) + " " + train["text"].fillna("").astype(str)
test["content"]  = test["title"].fillna("").astype(str)  + " " + test["text"].fillna("").astype(str)

# Remove duplicatas completas (title+text)
before = len(train)
train = train.drop_duplicates(subset=["title", "text"]).reset_index(drop=True)
print(f"Duplicatas removidas: {before - len(train)}")

X = train["content"].astype(str)
y = train["label"].astype(int)

print("Distribuição de classes (0=verdadeiro, 1=fake):")
print(y.value_counts())

# Split estratificado
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)
print("Train:", X_train.shape[0], "Val:", X_val.shape[0])

Duplicatas removidas: 501
Distribuição de classes (0=verdadeiro, 1=fake):
label
0    16996
1     5347
Name: count, dtype: int64
Train: 17874 Val: 4469


In [3]:
# 2) Pipeline: TF‑IDF word + TF‑IDF char + LinearSVC
# Word TF-IDF (conteúdo)
word_tfidf = TfidfVectorizer(
    stop_words="english",
    ngram_range=(1, 2),
    min_df=2,
    max_features=200000,
    sublinear_tf=True
)

# Char TF-IDF (estilo)
char_tfidf = TfidfVectorizer(
    analyzer="char_wb",      # n-grams de caracteres dentro de limites de palavra
    ngram_range=(3, 5),
    min_df=2,
    max_features=200000,
    sublinear_tf=True
)

features = FeatureUnion([
    ("word", word_tfidf),
    ("char", char_tfidf)
])

# Linear SVM (muito forte para texto)
clf = LinearSVC(
    C=2.0,
    class_weight="balanced",   # ajuda no desbalanceamento
    random_state=42
)

pipeline = Pipeline([
    ("features", features),
    ("clf", clf)
])

pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('word',
                                                 TfidfVectorizer(max_features=200000,
                                                                 min_df=2,
                                                                 ngram_range=(1,
                                                                              2),
                                                                 stop_words='english',
                                                                 sublinear_tf=True)),
                                                ('char',
                                                 TfidfVectorizer(analyzer='char_wb',
                                                                 max_features=200000,
                                                                 min_df=2,
                                                                 ngram_range=(3,
                                                                              5),
                                                                 sublinear_tf=True))])),
                ('clf',
                 LinearSVC(C=2.0, class_weight='balanced', random_state=42))])

In [4]:
# 3) Treinamento do modelo Linear SVM
# Para SVM,`decision_function` é usado como score e um threshold é ajustado para maximizar F1.

pipeline.fit(X_train, y_train)

# Scores contínuos (quanto maior, mais tende a classe 1)
val_scores = pipeline.decision_function(X_val)

# Busca de threshold para maximizar F1
thresholds = np.linspace(np.percentile(val_scores, 1), np.percentile(val_scores, 99), 200)
best_t = 0.0
best_f1 = -1.0

for t in thresholds:
    pred_t = (val_scores >= t).astype(int)
    f1 = f1_score(y_val, pred_t)
    if f1 > best_f1:
        best_f1 = float(f1)
        best_t = float(t)

val_pred = (val_scores >= best_t).astype(int)

# 4) Avaliação de desempenho (val)

print(f"Melhor threshold (val): {best_t:.4f}")
print(f"F1 (val) no melhor threshold: {best_f1:.6f}")
print("Classification report (val):")
print(classification_report(y_val, val_pred, digits=4))

Melhor threshold (val): -0.0591
F1 (val) no melhor threshold: 0.998129
Classification report (val):
              precision    recall  f1-score   support

           0     0.9994    0.9994    0.9994      3400
           1     0.9981    0.9981    0.9981      1069

    accuracy                         0.9991      4469
   macro avg     0.9988    0.9988    0.9988      4469
weighted avg     0.9991    0.9991    0.9991      4469



In [5]:
# Refit em todo o treino
pipeline.fit(X, y)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('word',
                                                 TfidfVectorizer(max_features=200000,
                                                                 min_df=2,
                                                                 ngram_range=(1,
                                                                              2),
                                                                 stop_words='english',
                                                                 sublinear_tf=True)),
                                                ('char',
                                                 TfidfVectorizer(analyzer='char_wb',
                                                                 max_features=200000,
                                                                 min_df=2,
                                                                 ngram_range=(3,
                                                                              5),
                                                                 sublinear_tf=True))])),
                ('clf',
                 LinearSVC(C=2.0, class_weight='balanced', random_state=42))])

In [6]:
# Salvamento do modelo treinado (artifact)
# Permite reprodutibilidade e geração de inferência sem novo treinamento

os.makedirs("artifacts", exist_ok=True)

joblib.dump(
    {
        "pipeline": pipeline,
        "best_threshold": best_t
    },
    "svm_word_char_linearsvc.joblib"
)

print("Artifact salvo em: svm_word_char_linearsvc.joblib")

Artifact salvo em: svm_word_char_linearsvc.joblib


In [7]:
#Inferência Final: Gera a submissão para o Kaggle utilizando o limiar (threshold) de classificação ótimo.

# Score no teste
test_scores = pipeline.decision_function(test["content"].astype(str))

test_pred = (test_scores >= best_t).astype(int)

submission = pd.DataFrame({
    "id": test["id"].values,
    "target": test_pred
})

submission.to_csv("submission_cpu_svm_word_char.csv", index=False)

print("Arquivo salvo em: submission_cpu_svm_word_char.csv")
print(submission.head())
print("Linhas:", len(submission))

Arquivo salvo em: submission_cpu_svm_word_char.csv
      id  target
0   5398       1
1   5503       1
2  23151       0
3  12669       0
4  27864       0
Linhas: 5712


#Conclusão:

O modelo baseado na combinação de TF-IDF de palavras e caracteres com Linear SVM apresentou desempenho extremamente elevado no conjunto de validação, alcançando F1-score de 0.998129 após ajuste do threshold, com métricas de precisão e recall próximas de 1 para ambas as classes. Esses resultados confirmam a eficácia da abordagem híbrida em capturar simultaneamente padrões semânticos e estilísticos relevantes para a tarefa de detecção de fake news.

Como próximo passo, será realizada uma exploração mais sistemática de hiperparâmetros, investigando diferentes configurações de n-grams e valores de regularização, com o objetivo de verificar possíveis ganhos adicionais de desempenho e robustez do modelo.